<a href="https://colab.research.google.com/github/tamara-kostova/MSc_Thesis_Neuroimaging/blob/master/07_sam3_adapter_finetune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SAM Adapter

[https://github.com/tianrun-chen/SAM-Adapter-PyTorch/tree/main](https://github.com/tianrun-chen/SAM-Adapter-PyTorch/tree/main)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
BASE_DIR = "/content/drive/MyDrive/MSc_Thesis_Neuroimaging"
SPLIT_DIR = f"{BASE_DIR}/data/split"

## Create masks

In [ ]:
import cv2
import numpy as np
from pathlib import Path

def create_tumor_masks(image_dir, output_dir):
    """Create binary tumor masks from MRI images"""
    image_path = Path(image_dir)
    output_path = Path(output_dir)
    output_path.mkdir(parents=True, exist_ok=True)
    print(f"Created output directory: {output_path}")
    for img_path in Path(image_dir).glob("*.jpg"):
        image = cv2.imread(str(img_path), cv2.IMREAD_GRAYSCALE)

        _, mask = cv2.threshold(
            image,
            cv2.THRESH_OTSU,
            255,
            cv2.THRESH_BINARY
        )

        kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
        mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)

        output_path = Path(output_dir) / f"{img_path.stem}_mask.png"
        print(f"saving to {output_path}")
        cv2.imwrite(str(output_path), mask)

create_tumor_masks(
    f"{SPLIT_DIR}/MRI_tumor_binary_norm/test/tumor",
    f"{BASE_DIR}/data/masks/tumor"
)


Created output directory: /content/drive/MyDrive/MSc_Thesis_Neuroimaging/data/masks/tumor
saving to /content/drive/MyDrive/MSc_Thesis_Neuroimaging/data/masks/tumor/y431_mask.png
saving to /content/drive/MyDrive/MSc_Thesis_Neuroimaging/data/masks/tumor/y224_mask.png
saving to /content/drive/MyDrive/MSc_Thesis_Neuroimaging/data/masks/tumor/y25_mask.png
saving to /content/drive/MyDrive/MSc_Thesis_Neuroimaging/data/masks/tumor/y1074_mask.png
saving to /content/drive/MyDrive/MSc_Thesis_Neuroimaging/data/masks/tumor/y720_mask.png
saving to /content/drive/MyDrive/MSc_Thesis_Neuroimaging/data/masks/tumor/y256_mask.png
saving to /content/drive/MyDrive/MSc_Thesis_Neuroimaging/data/masks/tumor/y1408_mask.png
saving to /content/drive/MyDrive/MSc_Thesis_Neuroimaging/data/masks/tumor/y684_mask.png
saving to /content/drive/MyDrive/MSc_Thesis_Neuroimaging/data/masks/tumor/y1064_mask.png
saving to /content/drive/MyDrive/MSc_Thesis_Neuroimaging/data/masks/tumor/y1236_mask.png
saving to /content/drive/My

## Create coco dataset

In [ ]:
import json
from pycocotools import mask as coco_mask
def create_coco_dataset(image_dir, mask_dir, output_file):
    image_path = Path(image_dir)
    mask_path = Path(mask_dir)

    print(f"📁 Images: {image_path} → {len(list(image_path.glob('*.jpg')))} files")
    print(f"📁 Masks: {mask_path} → {len(list(mask_path.glob('*.png')))} files")

    images = sorted(image_path.glob("*.jpg"))

    coco = {
        "info": {"description": "Medical Tumor Segmentation"},
        "images": [],
        "annotations": [],
        "categories": [{"id": 1, "name": "tumor"}]
    }

    annotation_id = 1
    image_id = 1
    successful = 0

    for img_file in images:
        mask_file = mask_path / f"{img_file.stem}_mask.png"

        if not mask_file.exists():
            print(f"❌ Missing: {mask_file}")
            continue

        try:
            coco["images"].append({
                "id": image_id,
                "file_name": img_file.name,
                "height": 224,
                "width": 224
            })

            mask = cv2.imread(str(mask_file), cv2.IMREAD_GRAYSCALE)
            if mask is None or mask.sum() == 0:
                image_id += 1
                continue

            mask_binary = (mask > 128).astype(np.uint8)
            rle = coco_mask.encode(np.asfortranarray(mask_binary))
            area = int(coco_mask.area(rle))
            rle_str = rle['counts'].decode('utf-8')

            coco["annotations"].append({
                "id": annotation_id,
                "image_id": image_id,
                "category_id": 1,
                "segmentation": {
                    "size": rle['size'],
                    "counts": rle_str
                },
                "area": area,
                "bbox": [0, 0, 224, 224],
                "iscrowd": 0
            })

            annotation_id += 1
            successful += 1

            if successful % 50 == 0:
                print(f"  ✅ {successful}...")

        except Exception as e:
            print(f"  ❌ {img_file.name}: {e}")

        image_id += 1

    with open(output_file, 'w') as f:
        json.dump(coco, f, indent=2)

    print(f"\n✅ COCO JSON SAVED!")
    print(f"   {len(coco['images'])} images")
    print(f"   {len(coco['annotations'])} annotations")
    print(f"   {output_file}")

create_coco_dataset(
    image_dir=f"{BASE_DIR}/data/split/MRI_tumor_binary_norm/test/tumor",
    mask_dir=f"{BASE_DIR}/data/masks/tumor",
    output_file=f"{BASE_DIR}/data/coco_tumor_binary.json"
)

with open(f"{BASE_DIR}/data/coco_tumor_binary.json") as f:
    coco = json.load(f)
    print(f"\n✅ VERIFIED: {len(coco['annotations'])} annotations in JSON")

📁 Images: /content/drive/MyDrive/MSc_Thesis_Neuroimaging/data/split/MRI_tumor_binary_norm/test/tumor → 225 files
📁 Masks: /content/drive/MyDrive/MSc_Thesis_Neuroimaging/data/masks/tumor → 225 files
  ✅ 50...
  ✅ 100...
  ✅ 150...
  ✅ 200...

✅ COCO JSON SAVED!
   225 images
   225 annotations
   /content/drive/MyDrive/MSc_Thesis_Neuroimaging/data/coco_tumor_binary.json

✅ VERIFIED: 225 annotations in JSON


## SAM Adapter

In [ ]:
!rm -rf /content/SAM-Adapter

In [ ]:
!git clone https://github.com/tianrun-chen/SAM-Adapter.git /content/SAM-Adapter
!cd /content/SAM-Adapter && ls -la

!ls /content/SAM-Adapter/

!cd /content/SAM-Adapter && pip install -r requirements.txt

!ls -la /content/SAM-Adapter/train.py

Cloning into '/content/SAM-Adapter'...
remote: Enumerating objects: 388, done.
remote: Counting objects: 100% (125/125), done.
remote: Compressing objects: 100% (78/78), done.
remote: Total 388 (delta 73), reused 47 (delta 47), pack-reused 263 (from 2)
Receiving objects: 100% (388/388), 10.48 MiB | 14.96 MiB/s, done.
Resolving deltas: 100% (107/107), done.
total 9740
drwxr-xr-x 6 root root    4096 Jan 13 20:56  .
drwxr-xr-x 1 root root    4096 Jan 13 20:56  ..
-rw-r--r-- 1 root root 2334033 Jan 13 20:56  Adaptor.pdf
drwxr-xr-x 2 root root    4096 Jan 13 20:56  configs
drwxr-xr-x 3 root root    4096 Jan 13 20:56  datasets
drwxr-xr-x 8 root root    4096 Jan 13 20:56  .git
-rw-r--r-- 1 root root       0 Jan 13 20:56  __init__.py
-rw-r--r-- 1 root root    1069 Jan 13 20:56  LICENSE
drwxr-xr-x 3 root root    4096 Jan 13 20:56  models
-rw-r--r-- 1 root root    5558 Jan 13 20:56  README.md
-rw-r--r-- 1 root root     388 Jan 13 20:56  requirements.txt
-rw-r--r-- 1 root root 4641040 Jan 13 20:5

In [ ]:
!pip install -q torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install -q imageio ipython matplotlib opencv-python PyYAML scikit-learn scipy tqdm numpy typing terminaltables Pillow tensorboardX onnxruntime setuptools timm easydict attr thop torchsummary

In [ ]:
!pip install -q -U openmim
!mim install -q mmcv


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.5/259.5 kB 8.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.1/50.1 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.7/89.7 kB 6.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 449.6/449.6 kB 23.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.7/52.7 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.5/314.5 kB 20.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.4/239.4 kB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.5/506.5 kB 32.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.1/77.1 kB 5.5 MB/s eta 0:00:0

In [ ]:
!cd /content/SAM-Adapter && python train.py \
  --config configs/sam3_adapter.yaml \
  --data_path /content/drive/MyDrive/MSc_Thesis_Neuroimaging/data/split/MRI_tumor_binary_norm/test/tumor \
  --mask_path /content/drive/MyDrive/MSc_Thesis_Neuroimaging/data/masks/tumor \
  --output_dir ./checkpoints/tumor \
  --num_epochs 5 \
  --batch_size 4 \
  --lr 1e-4

Traceback (most recent call last):
  File "/content/SAM-Adapter/train.py", line 10, in <module>
    import models
  File "/content/SAM-Adapter/models/__init__.py", line 2, in <module>
    from . import sam
  File "/content/SAM-Adapter/models/sam.py", line 10, in <module>
    from .mmseg.models.sam import ImageEncoderViT, MaskDecoder, TwoWayTransformer
  File "/content/SAM-Adapter/models/mmseg/__init__.py", line 1, in <module>
    import mmcv
ModuleNotFoundError: No module named 'mmcv'
